In [1]:
from Border import *
from BuildingBlock import *
from Coupler import *
from LayerCSEM import *
from nazca.interconnects import Interconnect



ImportError: attempted relative import with no known parent package

In [ ]:
import nazca as nd
# Layers as defined by CSEM
nd.add_layer(name='WG1', layer=11)


nd.add_layer(name='WG2_Opening', layer=25)
nd.add_layer(name='WG2', layer=21) #WG2_SE
nd.add_layer(name='MET1', layer=51)
nd.add_layer(name='CLD_OPEN', layer=60)
nd.add_layer(name='DEEP_GRID', layer=70)
nd.add_layer(name='TXT_LOGO', layer=22) # "WG2_SE_TXT"
nd.add_layer(name='EFF AREA', layer=2)
nd.add_layer(name='PAYLOAD', layer=0)

# Create a CROSS-SECTIONS for Passive (XS1) and Active (XS2)
nd.add_xsection(name='XS1')
nd.add_xsection(name='XS2')

# WAVEGUIDE PARAMETER RULES
width_WG = 0.8
rad_bend = 20

# Define the cross section to use for the PASSIVE WG
# Define PASSIVE interconnect according to waveguide parameter rules
nd.add_layer2xsection(xsection='XS1', layer='WG1')
nd.add_layer2xsection(xsection='XS2', layer='WG1')
nd.add_layer2xsection(xsection='XS2', layer='MET1')
passive = Interconnect(xs='XS1', width=width_WG, radius=rad_bend)
active = Interconnect(xs='XS2', width=width_WG, radius=rad_bend)





In [ ]:
# create xs + interconnect



# # create inverted xs + interconnect
# nd.add_xsection('xs1i')
# nd.add_layer2xsection(xsection='xs1i', layer='lay3', leftedge=(0.5, 0), rightedge=(0.5, 5))
# nd.add_layer2xsection(xsection='xs1i', layer='lay3', leftedge=(-0.5, 0), rightedge=(-0.5, -5))
# ici = nd.interconnects.Interconnect(xs='xs1i', radius=50, width=2.0)

def waveguide(length=30, posX = 0 , posY = 0):
    with nd.Cell(name=f'waveguide_{length}') as Waveguide:
        Waveguide.autobbox = True
        
        
        o1 = passive.strt(length=length, width=1, arrow=False).put(posX+60, posY+20)
        o2 = passive.strt(length=length, width=1, arrow=False).put(posX+60, posY-27)
        i1 = active.strt(length=5.0, arrow=False).put(posX+60)
        i2 = active.strt(length=5.0, arrow=False).put(posX+60, posY-50)
        
        # create layers in a xsection
        nd.add_layer2xsection(xsection='XS1', layer=1)
        nd.add_layer2xsection(xsection='XS1', layer=(2, 0), growx=2.0)
        nd.add_layer2xsection(xsection='XS1', layer=(3, 10), growx=5.0, accuracy=0.001)

        nd.strt(length=length, xs='XS1').put(posX+60, posY+42)

        
#                 # create layers in a xsection
#         nd.add_layer2xsection(xsection='XS2', layer=1)
#         nd.add_layer2xsection(xsection='XS2', layer=(2, 0), growx=2.0)
#         nd.add_layer2xsection(xsection='XS2', layer=(3, 10), growx=5.0, accuracy=0.001)

        nd.strt(length=length, xs='XS2').put(posX+60, posY-2)
#         nd.Pin('a0').put(0,0,180)
#         nd.Pin('a1').put(0,0,180)
#         nd.Pin('b0').put(length,0,0)
#         nd.Pin('b1').put(length,0,0)


        # add pins for connectivity:
        nd.Pin('a0', pin=o1.pin['a0']).put()
        nd.Pin('a1', pin=o2.pin['a0']).put()
        nd.Pin('b0', pin=o1.pin['b0']).put()
        nd.Pin('b1', pin=o2.pin['b0']).put()
        
        
#             nd.Pin('a0', pin==i1.pin['a0']).put()
#         nd.Pin('a1', pin=i2.pin['a0']).put()
#         nd.Pin('b0', pin=i1.pin['b0']).put()
#         nd.Pin('b1', pin=i2.pin['b0']).put()
#         nd.Pin('b2', pin=o1.pin['b0']).put()
#         nd.Pin('b1', pin=o2.pin['b0']).put()

        
        nd.put_stub() # show the pins in the layout

    return Waveguide

In [ ]:
# parametersBorder = {"chip_size_x": 2500, "chip_size_y": 2000}
# Border(**parametersBorder).put(-1250, -1000)
nd.nazca_logo(layers={'ring':1, 'box':20, 'bird':11}).put(-300, -300)

nd.image('UCL_logo.png').put(-700, -300, flop = True)
#nd.image('UCL_logo.png', threshold=0.6).put(-700, -300)
message = 'V.Liutarevich'
nd.text(text=message, height=30, layer=1, align='cc').put(-1000, -300)




def myMMI(length=30, posX = 0 , posY = 0):
    with nd.Cell(name=f'myMMI_{length}') as myMMI:
        myMMI.autobbox = False
        parametersDCoupler = {"waveguide_width": 2.0, "length": 32.704, "radius": 15, "spacing": 12, "gap": 0.8, "arrow": False}
        directionCoupler1 = DirectionalCoupler(**parametersDCoupler).put(-100, -20)
        waveguide1 = waveguide(length = 180).put(posX+70, posY+ 0)
        directionCoupler2 = DirectionalCoupler(**parametersDCoupler).put(posX+420, posY-20, flop=True)
        passive.sbend_p2p(pin1=directionCoupler1.pin['b1'], pin2=waveguide1.pin['a0']).put()
        passive.sbend_p2p(pin1=directionCoupler1.pin['b0'], pin2=waveguide1.pin['a1']).put()
        passive.sbend_p2p(pin1=waveguide1.pin['b0'], pin2=directionCoupler2.pin['b1']).put()
        passive.sbend_p2p(pin1=waveguide1.pin['b1'], pin2=directionCoupler2.pin['b0']).put()
    return myMMI

def main():
    my_mmi = myMMI(length = 400).put(-1000, -1000)

    nd.export_gds()
main()
